# Submodule 3: Data Integration

<img src="images/LessonPlan3.jpg" alt="Drawing" width=1000 />

## Overview & Purpose
## Integration of ChIP-seq, CUT&RUN, and CUT&Tag with each other and with RNA-seq and ATAC-seq
This submodule introduces concepts for integrating ChIP-seq, CUT&RUN, and CUT&Tag with RNA-seq and ATAC-seq.   

To demonstrate the process, we will build on the analysis performed in submodule 2, where we performed differential peak identification and created signal tracks.  

As a reminder, this module covers the processing of the data from three distinct but similar methods using downsampled data to improve runtime speed. The original data was published in :

Weber CM, et al. mSWI/SNF promotes Polycomb repression both directly and through genome-wide redistribution. Nat Struct Mol Biol. 2021  PMID: [34117481](https://pubmed.ncbi.nlm.nih.gov/34117481/)

Brahma S, Henikoff S. The BAF chromatin remodeler synergizes with RNA polymerase II and transcription factors to evict nucleosomes. Nat Genet. 2024 PMID: [38049663](https://pubmed.ncbi.nlm.nih.gov/38049663/)

We'll be comparing these data to RNA-seq and ATAC-seq in mESCs after baf inhibition published by Lurlaro M, et al. Mammalian SWI/SNF continuously restores local accessibility to chromatin. Nat Genet. 2021 PMID: [33558757](https://pubmed.ncbi.nlm.nih.gov/33558757/)

<div class="alert alert-block alert-warning" style="font-size:100%">
<span style="color:black"> We will be using processed files in this module. If you would like to know how to process RNA-seq or ATAC-seq data from scratch, please complete the other Sandbox modules.</span>
</div>

Note that to allow faster processing we have limited the reads to that of a single chromosome (chr4).  

### Ways to use this module
If you used submodule 1 or 2, you may recall how to navigate through the module. Throughout this module, we have color-coded commands according to ChIP-seq, CUT&RUN, and CUT&Tag. Therefore this module can be used to learn about the processing of each method individually, to compare each method to the others, or you can follow the colored commands to only process one type, either ChIP-seq, CUT&RUN, or CUT&Tag.
Commands for each method will be designated by an individual logo before the command, just like the following examples

<img src="images/ChIPseqLogo.jpg" alt="Drawing" style="width: 250px;" align="left"/>

In [ ]:
#run this cell for ChIP-seq
print("Code for ChIP-seq will be placed after the above image. Run these cells if performing ChIP-seq analysis.")

<img src="images/CUT&RUNLogo.jpg" alt="Drawing" style="width: 250px;" align="left"/>

In [ ]:
#run this cell for CUT&RUN
print("Code for CUT&RUN will be placed after the above image. Run these cells if performing CUT&RUN analysis.")

<img src="images/CUT&TagLogo.jpg" alt="Drawing" style="width: 250px;" align="left"/>

In [ ]:
#run this cell for CUT&Tag
print("Code for CUT&Tag will be placed after the above image. Run these cells if performing CUT&Tag analysis.")

<div class="alert alert-block alert-success" style="font-size:100%">
<span style="color:black"> By following the colors/images, you can run one, two, or all three types of analyses.</span>
</div>

### Required Files
In this stage of the module, you will use the sam files that are the output from submodule 1 (We also provide them if you skipped submodule 1). You can also use this module on your own data or any published ChIP-seq, CUT&RUN, or CUT&Tag dataset. 

<div class="alert-info" style="font-size:200%">
STEP 1: Set Up Environment
</div>

Initial items to configure your Cloud environment. In this step we will use conda to install the following packages:

[deeptools](https://deeptools.readthedocs.io/en/develop/)

[bedtools](https://bedtools.readthedocs.io/en/latest/)

In [ ]:
#First let's install mamba to configure our environment
! curl -L -O "https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-$(uname)-$(uname -m).sh"
! bash Miniforge3-$(uname)-$(uname -m).sh -u -b -p $HOME/mambaforge
print("done")

In [ ]:
#now let's install several required packages
!mamba install -c bioconda deeptools bedtools -y
!pip install jupyterquiz==2.0.7 jupytercards
#!pip install --user igv-notebook
print("done")

In [ ]:
#Now let's import packages that we installed
numthreads=!lscpu | grep '^CPU(s)'| awk '{print $2-1}'
numthreadsint = int(numthreads[0])
import sys
import os
from jupyterquiz import display_quiz
from IPython.display import IFrame
#from IPython.display import display
from IPython.display import Image
from jupytercards import display_flashcards
#import igv_notebook
import pandas as pd
#import modules for matching-type quiz
%cd questions
from quiz_module import run_quiz
%cd ../
import json
import ipywidgets as widgets
from IPython.display import display
import random
print("done")

In [ ]:
wd="~/SageMaker/SandboxChromatinOccupancy"
%cd $wd
#show which folder you are working in. 
!pwd

In [ ]:
# These commands move into our Tutorial 1 directory and create our subdirectory structure.
!mkdir -p $wd/Submodule3/
%cd $wd/Submodule3/
!mkdir -p $wd/Submodule3/RNAseq
!mkdir -p $wd/Submodule3/ATACseq

In [ ]:
#Let's copy and extract our tutorial files
!wget https://chromatinoccupancytutorial.s3.us-east-2.amazonaws.com/Submodule3.zip
!unzip Submodule3.zip

In [ ]:
#you should see several of the files we created in previous submodules as well as some files from ATAC-seq and RNA-seq that we'll be using to compare.
!ls Submodule3_Inputfiles

<div class="alert-info" style="font-size:200%">
Submodule 3.2: Comparison to ATAC-seq
</div>

In submodule 2, we performed peak identification and obtained lists of differential peaks. In this module let's test grab see if our differential peaks correspond to sites of differential accessibility, using ATAC-seq.

We've prepared the ATAC-seq data for you. To learn how to process ATAC-seq, please visit the relevant NIH/NIGMS Sandbox lesson: [CloudATAC](https://github.com/NIGMS/ATAC-Seq-and-Single-Cell-ATAC-Seq-Analysis).

<img src="images/cloudATAC.png" alt="Drawing" style="width: 200px;"/>

The ATAC-seq data we'll use compares accessibility in control mESCs (ctl) to that after baf inhibition (bafi). Similar to how we've processed data in this module, the prepped ATAC-seq data has signal tracks for each sample in bigwig format as well as differential accessibility in bed format.

In [ ]:
#Let's list the relevant files by using a wildcard:
!ls $wd/Submodule3/Submodule3_Inputfiles/*ATAC*

First, let's plot the average ATAC-seq signal at differential ChIP-seq, CUT&RUN, or CUT&Tag peaks. We'll use deeptools functions computeMatrix and plotProfile, similar to what we did in Submodule 2. However, this time we'll specific the ATAC-seq files as the signal (-S option) in order to plot accessibility across differential ChIP-seq, CUT&RUN, or CUT&Tag peaks (which will be specificied by the -R option).

<img src="images/ChIPseqLogo.jpg" alt="Drawing" style="width: 100px;" align="left" /> Run the following command for ChIP-seq.

In [ ]:
#First compute the signal per region matrix.
!computeMatrix reference-point --referencePoint center -S $wd/Submodule3/Submodule3_Inputfiles/ctl_ATAC.bw $wd/Submodule3/Submodule3_Inputfiles/bafi_ATAC.bw -R $wd/Submodule3/Submodule3_Inputfiles/H3K27ac_higher_in_auxin.bed $wd/Submodule3/Submodule3_Inputfiles/H3K27ac_higher_in_noauxin.bed -o $wd/Submodule3/ATACseq/ATAC_on_diffChIPseqprofileMatrix -a 4000 -b 4000
!plotProfile -m $wd/Submodule3/ATACseq/ATAC_on_diffChIPseqprofileMatrix -o $wd/Submodule3/ATACseq/ATAC_on_diffChIPseqprofile.png --perGroup

Let's view the output

In [ ]:
%cd $wd/Submodule3
Image(url= "Submodule3/ATACseq/ATAC_on_diffChIPseqprofile.png", width=800, height=800)

In [ ]:
#We can also create a heatmap using the same matrix we created from the computeMatrix command. The command is similar to plotProfile.
!plotHeatmap -m $wd/Submodule3/ATACseq/ATAC_on_diffChIPseqprofileMatrix -o $wd/Submodule3/ATACseq/ATAC_on_diffChIPseqheatmap.png --perGroup --colorMap Reds --whatToShow "heatmap and colorbar"
%cd $wd/Submodule3
Image(url= "Submodule3/ATACseq/ATAC_on_diffChIPseqheatmap.png", width=200, height=200)

Remember that we are running on very few regions and downsampled data for the sake of this tutorial, so the results are not as clean as a genome-wide analysis. Regardless, these figures demonstrate that the differential H3K27ac peaks do not correspond to pronounced changes in accessibility after baf inhibition (compare ctl to bafi in each category). They do show, however, that sites with H3K27ac higher in auxin are more accessible than sites with H3K27ac higher in no auxin.

<div class="alert-info" style="font-size:100%">
Next, let's do the opposite to ask the question: Do differentially accessible sites, have differential occupancy? We can do so by plotting the occupancy signal (bigwig) from ChIP-seq, CUT&RUN, or CUT&Tag across differentially accessible regions (bed). This time we'll do both the plotProfile and plotHeatmap command in the same cell.
</div>

<img src="images/ChIPseqLogo.jpg" alt="Drawing" style="width: 100px;" align="left" /> Run the following command for ChIP-seq.

In [ ]:
#First compute the signal per region matrix.
!computeMatrix reference-point --referencePoint center -S $wd/Submodule3/Submodule3_Inputfiles/H3K27ac_ChIPseq_noauxdedup.bw $wd/Submodule3/Submodule3_Inputfiles/H3K27ac_ChIPseq_auxdedup.bw -R $wd/Submodule3/Submodule3_Inputfiles/IncreasedinbafiATACpeaks.bed $wd/Submodule3/Submodule3_Inputfiles/DecreasedinbafiATACpeaks.bed -o $wd/Submodule3/ATACseq/ChIPseq_on_diffATACseqprofileMatrix -a 4000 -b 4000
!plotProfile -m $wd/Submodule3/ATACseq/ChIPseq_on_diffATACseqprofileMatrix -o $wd/Submodule3/ATACseq/ChIPseq_on_diffATACseqprofile.png --perGroup
#We can also create a heatmap using the same matrix we created from the computeMatrix command. The command is similar to plotProfile.
!plotHeatmap -m $wd/Submodule3/ATACseq/ChIPseq_on_diffATACseqprofileMatrix -o $wd/Submodule3/ATACseq/ChIPseq_on_diffATACseqheatmap.png --perGroup --colorMap Reds --whatToShow "heatmap and colorbar"

Let's view the profile:

In [ ]:
%cd $wd/Submodule3
Image(url= "Submodule3/ATACseq/ChIPseq_on_diffATACseqprofile.png", width=800, height=800)

In [ ]:
#and the heatmap
Image(url= "Submodule3/ATACseq/ChIPseq_on_diffATACseqheatmap.png", width=200, height=200)

<div class="alert-info" style="font-size:100%">
Let's say, instead of just plotting the signal across all these sites, we want to directly identify if any peaks are changed in both ATAC-seq and ChIP-seq, CUT&RUN, or CUT&Tag. ncy signal (bigwig) from ChIP-seq, CUT&RUN, or CUT&Tag across differentially accessible regions (bed). We can do so by intersecting the peaks in each file.
</div>
<img src="images/bedtoolsintersect.png" alt="Drawing" style="width: 300px;" align="left" />

In [ ]:
#intersect and count how many H3K27ac higer in auxin have decreased accessibility in bafi.
!intersectBed -u -a $wd/Submodule3/Submodule3_Inputfiles/H3K27ac_higher_in_auxin.bed -b $wd/Submodule3/Submodule3_Inputfiles/DecreasedinbafiATACpeaks.bed | wc -l

<div class="alert-success" style="font-size:100%">
This showed that only 5 of the peaks in "H3K27ac_higher_in_auxin.bed" overlapped that of "DecreasedinbafiATACpeaks.bed". (Remember that these are severely downsampled files.)
How do we know if that overlap is more than you would expect by chance?
</div>

In [ ]:
#We can take a set of random unchanged peaks to see if we get the same overlap. First let's count how many DecreasedinbafiATACpeaks.bed there are.
!wc -l $wd/Submodule3/Submodule3_Inputfiles/DecreasedinbafiATACpeaks.bed

In [ ]:
#The previous command showed that there are 442 Decreased peaks that we used in the intersection analysis. Let's grab the same number of randomly selected unchanged peaks.
!shuf -n 442 $wd/Submodule3/Submodule3_Inputfiles/UnchangedATACpeaks.bed > $wd/Submodule3/ATACseq/randomATACpeaks.bed
#Now let's do the intersection!
!intersectBed -u -a $wd/Submodule3/Submodule3_Inputfiles/H3K27ac_higher_in_auxin.bed -b $wd/Submodule3/ATACseq/randomATACpeaks.bed | wc -l

<div class="alert-warning" style="font-size:200%">Note: the number resulting from the above command will change every time because of the randomization.</div>

How do we find out if the observed overlap is significantly higher than random loci?

<img src="images/permutationexample.gif" alt="Drawing" style="width: 500px;" align="left" />

Permutation testing is a good way to test if your overlap is more than what you might expect from random peaks. This method can be used to derive the expected value as well as a significance score for the overlap. The expected value is the average overlap in your randomized permutations. The significance is the number of permutations that matched or were higher than your observed overlap out of the total number of permutations that you ran. For example if you ran 1000 permutations and only one was equal to your observed overlap, we can represent the p-value as 1/1000 (p=.001). 

Notice that your minimum p-value is defined by the number of permutations that you run (i.e. you cannot get a p-value of 0.001 if you only ran 100 permutations).

In [ ]:
#Let's try it out by running the randomization 100 times! 
#First let's make an empty list to which we'll append the results of each permutation
randomoverlaps=[]
NumberofPermutations=100

#Now let's do the randomization and overlap 100 times
import statistics
permutation=1
while permutation <= NumberofPermutations:
    overlap=!shuf -n 442 $wd/Submodule3/Submodule3_Inputfiles/UnchangedATACpeaks.bed | intersectBed -u -a $wd/Submodule3/Submodule3_Inputfiles/H3K27ac_higher_in_auxin.bed -b stdin | wc -l
    randomoverlaps.append(int(overlap[0]))
    permutation+=1

#Now let's get the average overlap in the permutations
print("mean of the permutations is: " + str(statistics.mean(randomoverlaps)))

#And get the p-value by counting how many permutations had an overlap of at least 5
permcount=len([permut for permut in randomoverlaps if permut >= 5])
print("count of permutations equal or higher than observed: " + str(permcount))
print("therefore, the p-value using " + str(NumberofPermutations) + " permutations is: " + str(permcount/NumberofPermutations))

<div class="alert-warning" style="font-size:100%">Note: the exact p-values will change every time because of the randomization, however they should hover around the approximately the same. Permutation testing works better with a higher number of peaks than what we used in our example (due to downsampling of the data).</div>

<div class="alert-info" style="font-size:200%">
Interactive Quiz Question: Click on the correct answer in following cell.
</div>

In [ ]:
#Run for the quiz
%cd $wd/Submodule3
run_quiz("../questions/PermutationTesting.json", instant_feedback=True, shuffle_questions=False, shuffle_answers=True)

<div class="alert-info" style="font-size:200%">
Submodule 3.3: Comparison to RNA-seq
</div>

Now, Let's see if our differential peaks correspond to differentially expressed genes.

We've prepared the RNA-seq data for you. To learn how to process RNA-seq, please visit the relevant NIH/NIGMS Sandbox lesson: [RNA-seq](https://github.com/NIGMS/RNA-Seq-Differential-Expression-Analysis).

<img src="images/SandBoxRNAseq.png" alt="Drawing" style="width: 200px;"/>

Let's start this analysis after calling differentially expressed genes (DEGs).

In [ ]:
#Let's get the genomic coordinates of these genes by matching the names of DEGs to those of a total list of genes and their coordinates from a bed file.
%cd $wd/Submodule3

#First, let's make sure the DEGs list is sorted.
!sort -k 1b,1b --stable $wd/Submodule3/Submodule3_Inputfiles/degs.txt > $wd/Submodule3/RNAseq/degs_sorted.txt

#Next let's also sort the list of total genes. Note that in this file that gene names are in the 5th column:
!sort -k 5b,5b --stable $wd/Submodule3/Submodule3_Inputfiles/chr4genes_mm39rnaseq.bed > $wd/Submodule3/RNAseq/allgenes_sorted.txt

#Now let's use the join function to only keep genes that are in my list of degs. Note that join replaces tabs with spaces, so we'll also make sure the output is tab separated using the sed function.
#We use -1 1 to denote that the gene names are in column #1 of our first file, and we use -2 5 to denote that the gene names are in column #5 of our second file.
!join -1 1 -2 5 $wd/Submodule3/RNAseq/degs_sorted.txt $wd/Submodule3/RNAseq/allgenes_sorted.txt | sed -e 's/ /\t/g' > $wd/Submodule3/RNAseq/degs_coordinates.txt


In [ ]:
#Let's look at the first 10 lines of the resulting file:
!head $wd/Submodule3/RNAseq/degs_coordinates.txt

Now that we have a file with the coordinates of our differentially expressed genes, we can reformat it so that we can plot our ChIP-seq, CUT&RUN, or CUT&Tag signal near these genes.
Let's reformat as a bed format: [bed](http://useast.ensembl.org/info/website/upload/bed.html)

In [ ]:
%cd $wd/Submodule3
#Use awk to rearrange and fill in empty columns.
!awk '{print $2"\t"$3"\t"$4"\t"$1"\t255\t"$5}' RNAseq/degs_coordinates.txt > RNAseq/degs_coordinates.bed

#Now use deeptools computeMatrix, but this time we'll use the scale-regions function. Note the use of -m this tim to specify how big to plot the inside of the gene.
!computeMatrix scale-regions -S $wd/Submodule3/Submodule3_Inputfiles/H3K27ac_ChIPseq_noauxdedup.bw $wd/Submodule3/Submodule3_Inputfiles/H3K27ac_ChIPseq_auxdedup.bw -R $wd/Submodule3/RNAseq/degs_coordinates.bed -o $wd/Submodule3/RNAseq/degProfileMatrix -a 4000 -b 4000 -m 4000
!plotProfile -m $wd/Submodule3/RNAseq/degProfileMatrix -o $wd/Submodule3/RNAseq/ChIPseq_on_degs_profile.png --perGroup

In [ ]:
%cd $wd/Submodule3
Image(url= "Submodule3/RNAseq/ChIPseq_on_degs_profile.png", width=400, height=400)

<div class="alert-info" style="font-size:100%">
Let's say, instead of just plotting the signal across all these sites, we want to directly extract genes that have differential occupancy near the TSS. Similar to above we can use bedtools intersect, but first we need to create a file with the region around the TSS that we want to test.

However, for genes the strand matters! For genes that are on the + strand, the TSS is in the 2nd column. But, for gene on the - strand, the TSS is in the 3rd column! 
</div>
<img src="images/BEDgeneorientation.jpg" alt="Drawing" style="width: 500px;" align="left" />

</b>
When we used deeptools, it read the strand from the 6th column to correctly orient the genes. Below, we'll take 500 bp around the TSS by integrating the strand information.

In [ ]:
#create a file with the TSS and 250 bp upstream an downstream.
%cd $wd/Submodule3
!awk '{if ($6 == "+") print $1"\t"$2-250"\t"$2+250"\t"$4"\t"$5"\t"$6; else print $1"\t"$3-250"\t"$3+250"\t"$4"\t"$5"\t"$6}' RNAseq/degs_coordinates.bed > RNAseq/degs_TSS.bed

#Prep the total H3K27ac peaks from the xls file produced by manorm in Submodule2, by removing the header and keeping only columns 1-3.
!cat $wd/Submodule3/Submodule3_Inputfiles/H3K27ac_noauxin_vs_H3K27ac_auxin_all_MAvalues.xls | grep -v start | cut -f 1-3 > $wd/Submodule3/RNAseq/TotalChIPseqPeaks.txt

#Now use intersectBed with our total peaks.
!intersectBed -u -a $wd/Submodule3/RNAseq/degs_TSS.bed -b $wd/Submodule3/RNAseq/TotalChIPseqPeaks.txt > $wd/Submodule3/RNAseq/degs_withH3K27ac.bed

#let's look at the first 10 lines
!head $wd/Submodule3/RNAseq/degs_withH3K27ac.bed

#Let's also count how many:
!wc -l $wd/Submodule3/RNAseq/degs_withH3K27ac.bed

### Similar to above, we can do permutation testing to see if this overlap is significant:

In [ ]:
!wc -l RNAseq/degs_TSS.bed

In [ ]:
#First let's reformat our total genes list to follow the 6 column format, and get only 250 bp on either side of the TSS.
%cd $wd/Submodule3
!cat Submodule3_Inputfiles/chr4genes_mm39rnaseq.bed | awk '{print $1"\t"$2"\t"$3"\t"$5"\t255\t"$4}' | awk '{if ($6 == "+") print $1"\t"$2-250"\t"$2+250"\t"$4"\t"$5"\t"$6; else print $1"\t"$3-250"\t"$3+250"\t"$4"\t"$5"\t"$6}' > RNAseq/total_TSSs.bed
#First let's make an empty list to which we'll append the results of each permutation
randomoverlaps=[]
NumberofPermutations=100

#Now let's do the randomization and overlap 100 times. There were 45 degs in our list, so lets take the same number of random genes.
import statistics
permutation=1
while permutation <= NumberofPermutations:
    overlap=!shuf -n 45 RNAseq/total_TSSs.bed | intersectBed -u -a stdin -b $wd/Submodule3/RNAseq/TotalChIPseqPeaks.txt | wc -l
    randomoverlaps.append(int(overlap[0]))
    permutation+=1

#Now let's get the average overlap in the permutations
print("mean of the permutations is: " + str(statistics.mean(randomoverlaps)))

#And get the p-value by counting how many permutations had an overlap of at least 13, which was our observed value
permcount=len([permut for permut in randomoverlaps if permut >= 13])
print("count of permutations equal or higher than observed: " + str(permcount))
print("therefore, the p-value using " + str(NumberofPermutations) + " permutations is: " + str(permcount/NumberofPermutations))

In [ ]:
%cd $wd
#Run for the quiz
run_quiz("./questions/geneformat.json", instant_feedback=True, shuffle_questions=True, shuffle_answers=True)

<div class="alert alert-block alert-success" style="font-size:120%">
<span style="color:black">Congrats! You have successfully performed some analysis and visualization comparison of ATAC-seq and RNA-seq to that of chromatin occupancy obtained by ChIP-seq, CUT&RUN, or CUT&Tag. Thank you for completing this module!</span></div>